In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [82.8 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,866 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages

In [2]:
df = spark.read.csv("PySpark_DataOriginal.csv",header=True)

In [3]:
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [5]:
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [6]:
from pyspark.sql.types import *
schema = StructType([
    StructField('id', IntegerType()),
    StructField('first_name', StringType()),
    StructField('last_name',StringType()),
    StructField('gender',StringType()),
    StructField('City',StringType()),
    StructField('JobTitle',StringType()),
    StructField('Salary',IntegerType()),
    StructField('Latitude',FloatType()),
    StructField('Longitude',FloatType()),
])

In [7]:
df1 = spark.read.csv("PySpark_DataOriginal.csv", header=True, schema=schema)
df1.show()

+---+----------+----------+------+---------------+--------------------+------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|  null| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|  null|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|  null| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|  null| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|  null|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|  null| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|                null|  null| 24.879416|11

In [8]:
df1.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'int'),
 ('Latitude', 'float'),
 ('Longitude', 'float')]

In [25]:
from pyspark.sql.functions import col, regexp_replace
df2 = df1.withColumn("City", regexp_replace(col("City"), " ", ","))

In [26]:
df2.show()

+---+----------+----------+------+---------------+--------------------+------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa,Ruda| Assistant Professor|  null| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|  null|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|  null| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|  null| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|  null|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|  null| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|                null|  null| 24.879416|11

In [27]:
from pyspark.sql.functions import col, regexp_replace
df2 = df2.withColumn("City", regexp_replace(col("City"), ",", " "))

In [28]:
df2.show()

+---+----------+----------+------+---------------+--------------------+------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|  null| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|  null|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|  null| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|  null| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|  null|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|  null| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|                null|  null| 24.879416|11